### IMPORT - Modules and raw data

In [1]:
import pandas as pd
pd.set_option('precision', 4)

stations = pd.read_csv('/content/drive/MyDrive/Data/raw/weather/weather_stations.csv', encoding="latin-1",header=2)
stations["Station ID"] = stations["Station ID"]

In [2]:
active_stations = stations[stations["Last Year"] == 2021]
all_stations_list = active_stations["Station ID"].to_list()

bc_stations = stations[stations["Province"] == 'BRITISH COLUMBIA']
station_id_list = bc_stations["Station ID"].to_list()

This code block takes a few minutes to run. It grabs all records from 2021 for stations across Canada.

In [ ]:
li = []

for station_id in all_stations_list:
    url = 'https://climate.weather.gc.ca/climate_data/bulk_data_e.html?format=csv&stationID=' + str(station_id) + '&Year=2021&timeframe=2'
    df = pd.read_csv(url)
    df.insert(0, "Station ID", station_id)
    li.append(df)
  
all_stations_raw = pd.concat(li, axis=0, ignore_index=True)
all_stations_raw.to_csv('/content/drive/MyDrive/Data/raw/weather/all_stations_raw.csv')

Run it once, then run this block instead.

In [ ]:
raw = pd.read_csv('/content/drive/MyDrive/Data/raw/weather/all_stations_raw.csv')

display(raw.head(5))

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5,10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,Station ID,Longitude (x),Latitude (y),Station Name,Climate ID,Date/Time,Year,Month,Day,Data Quality,Max Temp (°C),Max Temp Flag,Min Temp (°C),Min Temp Flag,Mean Temp (°C),Mean Temp Flag,Heat Deg Days (°C),Heat Deg Days Flag,Cool Deg Days (°C),Cool Deg Days Flag,Total Rain (mm),Total Rain Flag,Total Snow (cm),Total Snow Flag,Total Precip (mm),Total Precip Flag,Snow on Grnd (cm),Snow on Grnd Flag,Dir of Max Gust (10s deg),Dir of Max Gust Flag,Spd of Max Gust (km/h),Spd of Max Gust Flag
0,0,26,-123.74,48.94,CHEMAINUS,1011500,2021-01-01,2021,1,1,†,9.5,NaN,7.0,NaN,8.3,NaN,9.7,NaN,0.0,NaN,27.2,NaN,0.0,NaN,27.2,NaN,0.0,NaN,NaN,NaN,NaN,NaN
1,1,26,-123.74,48.94,CHEMAINUS,1011500,2021-01-02,2021,1,2,†,8.5,NaN,7.0,NaN,7.8,NaN,10.2,NaN,0.0,NaN,41.9,NaN,0.0,NaN,41.9,NaN,0.0,NaN,NaN,NaN,NaN,NaN
2,2,26,-123.74,48.94,CHEMAINUS,1011500,2021-01-03,2021,1,3,†,8.0,NaN,3.0,NaN,5.5,NaN,12.5,NaN,0.0,NaN,29.8,NaN,0.0,NaN,29.8,NaN,0.0,NaN,NaN,NaN,NaN,NaN
3,3,26,-123.74,48.94,CHEMAINUS,1011500,2021-01-04,2021,1,4,†,8.5,NaN,5.0,NaN,6.8,NaN,11.2,NaN,0.0,NaN,7.8,NaN,0.0,NaN,7.8,NaN,0.0,NaN,NaN,NaN,NaN,NaN
4,4,26,-123.74,48.94,CHEMAINUS,1011500,2021-01-05,2021,1,5,†,6.5,NaN,3.0,NaN,4.8,NaN,13.2,NaN,0.0,NaN,44.0,NaN,0.0,NaN,44.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN


### 2021/11/16 - Rain by station for Nov 13-15

In [ ]:
filtered = raw[["Station ID", "Climate ID", "Station Name", "Longitude (x)", "Latitude (y)", "Date/Time", "Month", "Day", "Total Precip (mm)"]]
filtered = filtered[filtered["Month"] == 11]

nov13 = filtered[filtered["Day"] == 13].rename(columns={"Total Precip (mm)": "Nov 13 Rainfall"})
nov14 = filtered[filtered["Day"] == 14].rename(columns={"Total Precip (mm)": "Nov 14 Rainfall"})
nov15 = filtered[filtered["Day"] == 15].rename(columns={"Total Precip (mm)": "Nov 15 Rainfall"})

step1 = (pd
            .merge(left=nov13,right=nov14[["Station ID", "Nov 14 Rainfall"]], left_on="Station ID", right_on="Station ID"))

step2 = (pd
            .merge(left=step1,right=nov15[["Station ID", "Nov 15 Rainfall"]], left_on="Station ID", right_on="Station ID")
            .dropna(subset=["Nov 13 Rainfall", "Nov 14 Rainfall", "Nov 15 Rainfall"], how='all')
            .sort_values('Nov 14 Rainfall', ascending=False))

display(step2)
step2.to_csv('/content/drive/MyDrive/Data/exports/weather_station_data_nov14and15.csv')

,Station ID,Climate ID,Station Name,Longitude (x),Latitude (y),Date/Time,Month,Day,Nov 13 Rainfall,Nov 14 Rainfall,Nov 15 Rainfall
946,833,1105669,N VANCOUVER WHARVES,-123.12,49.31,2021-11-13,11,13,38.8,288.2,43.0
1110,50517,1113543,HOPE AIRPORT,-121.49,49.37,2021-11-13,11,13,17.1,174.0,103.5
72,82,1016335,PORT RENFREW,-124.33,48.59,2021-11-13,11,13,120.8,164.0,NaN
858,50368,1102221,CULTUS LAKE,-121.98,49.08,2021-11-13,11,13,20.2,133.2,74.6
923,810,1105192,MISSION WEST ABBEY,-122.27,49.15,2021-11-13,11,13,52.4,132.8,15.2
...,...,...,...,...,...,...,...,...,...,...,...
755,650,1097646,SPOKIN LAKE 4E,-121.69,52.18,2021-11-13,11,13,0.0,NaN,9.6
1109,49588,1113542,HOPE A,-121.50,49.37,2021-11-13,11,13,16.9,NaN,NaN
1222,1046,1126070,PEACHLAND,-119.72,49.78,2021-11-13,11,13,17.0,NaN,NaN
1378,1180,1152850,FERNIE,-115.07,49.49,2021-11-13,11,13,NaN,NaN,15.6


### 2021/11/16 - North Vancouver time series of Nov. 14 and 15.

This block takes a few minutes to run, so only run it if you need to.

In [ ]:
lis = []

for year in range(1980, 2022):
  url = 'https://climate.weather.gc.ca/climate_data/bulk_data_e.html?format=csv&stationID=833&Year=' + str(year) + '&Month=11&Day=14&timeframe=2'
  df = pd.read_csv(url)
  lis.append(df)
  
northvan = pd.concat(lis, axis=0, ignore_index=True)
northvan.to_csv('/content/drive/MyDrive/Data/raw/weather/north_van_historical.csv')

display(northvan)
northvan.to_csv('/content/drive/MyDrive/Data/exports/rain_van.csv')

,Longitude (x),Latitude (y),Station Name,Climate ID,Date/Time,Year,Month,Day,Data Quality,Max Temp (°C),Max Temp Flag,Min Temp (°C),Min Temp Flag,Mean Temp (°C),Mean Temp Flag,Heat Deg Days (°C),Heat Deg Days Flag,Cool Deg Days (°C),Cool Deg Days Flag,Total Rain (mm),Total Rain Flag,Total Snow (cm),Total Snow Flag,Total Precip (mm),Total Precip Flag,Snow on Grnd (cm),Snow on Grnd Flag,Dir of Max Gust (10s deg),Dir of Max Gust Flag,Spd of Max Gust (km/h),Spd of Max Gust Flag
0,-123.12,49.31,N VANCOUVER WHARVES,1105669,1980-01-01,1980,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.8,NaN,0.0,NaN,0.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-123.12,49.31,N VANCOUVER WHARVES,1105669,1980-01-02,1980,1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.6,NaN,0.0,NaN,7.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-123.12,49.31,N VANCOUVER WHARVES,1105669,1980-01-03,1980,1,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.8,NaN,0.0,NaN,0.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-123.12,49.31,N VANCOUVER WHARVES,1105669,1980-01-04,1980,1,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.6,NaN,0.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-123.12,49.31,N VANCOUVER WHARVES,1105669,1980-01-05,1980,1,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15336,-123.12,49.31,N VANCOUVER WHARVES,1105669,2021-12-27,2021,12,27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15337,-123.12,49.31,N VANCOUVER WHARVES,1105669,2021-12-28,2021,12,28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15338,-123.12,49.31,N VANCOUVER WHARVES,1105669,2021-12-29,2021,12,29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15339,-123.12,49.31,N VANCOUVER WHARVES,1105669,2021-12-30,2021,12,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
nov14 = northvan[northvan["Date/Time"].str.contains("11-14")]
nov15 = northvan[northvan["Date/Time"].str.contains("11-15")]
nov = pd.concat([nov14, nov15])
nov_sum = nov[['Station Name', 'Year', 'Total Rain (mm)']].groupby(['Station Name', 'Year']).sum().reset_index()

display(nov_sum)

,Station Name,Year,Total Rain (mm)
318,N VANCOUVER WHARVES,1980,1.0
683,N VANCOUVER WHARVES,1981,12.0
1048,N VANCOUVER WHARVES,1982,NaN
1413,N VANCOUVER WHARVES,1983,61.4
1779,N VANCOUVER WHARVES,1984,0.0
...,...,...,...
13833,N VANCOUVER WHARVES,2017,NaN
14198,N VANCOUVER WHARVES,2018,19.2
14563,N VANCOUVER WHARVES,2019,14.0
14929,N VANCOUVER WHARVES,2020,4.0


,Station Name,Year,Total Rain (mm)
0,N VANCOUVER WHARVES,1980,1.0
1,N VANCOUVER WHARVES,1981,22.0
2,N VANCOUVER WHARVES,1982,0.0
3,N VANCOUVER WHARVES,1983,104.2
4,N VANCOUVER WHARVES,1984,0.0
5,N VANCOUVER WHARVES,1985,21.8
6,N VANCOUVER WHARVES,1986,9.7
7,N VANCOUVER WHARVES,1987,26.8
8,N VANCOUVER WHARVES,1988,5.3
9,N VANCOUVER WHARVES,1989,3.8


### 2021/11/24 - Average November rainfall for North Vancouver

In [ ]:
northvan_months = northvan[["Year", "Month", "Total Rain (mm)"]].groupby(["Year", "Month"]).sum().reset_index()
northvan_novembers = northvan_months[northvan_months["Month"] == 11]

display(northvan_novembers)

,Year,Month,Total Rain (mm)
10,1980,11,372.2
22,1981,11,246.0
34,1982,11,0.0
46,1983,11,412.2
58,1984,11,412.9
70,1985,11,100.0
82,1986,11,280.7
94,1987,11,210.9
106,1988,11,304.6
118,1989,11,319.2


### 2021/11/24 - Temperatures in cities across Canada

This takes ~3 minutes to run, so you may not want to run it every time.

In [ ]:
cities = [
          "48549", #Toronto
          "51442", #Vancouver
          "51097", #Winnipeg
          "50149", #Edmonton
          "53938", #Halifax
          "51157", #Montreal
          ]

lis = []

for city in cities:
  for year in range(1980, 2022):
    url = 'https://climate.weather.gc.ca/climate_data/bulk_data_e.html?format=csv&stationID=' + city + '&Year=' + str(year) + '&timeframe=2'
    df = pd.read_csv(url)
    df["Station ID"] = city
    lis.append(df)

all_cities = pd.concat(lis, axis=0, ignore_index=True)
display(all_cities)
all_cities.to_csv('/content/drive/MyDrive/Data/raw/weather/cities_weather_data.csv')

all_cities = pd.read_csv('/content/drive/MyDrive/Data/raw/weather/cities_weather_data.csv')

,Longitude (x),Latitude (y),Station Name,Climate ID,Date/Time,Year,Month,Day,Data Quality,Max Temp (°C),Max Temp Flag,Min Temp (°C),Min Temp Flag,Mean Temp (°C),Mean Temp Flag,Heat Deg Days (°C),Heat Deg Days Flag,Cool Deg Days (°C),Cool Deg Days Flag,Total Rain (mm),Total Rain Flag,Total Snow (cm),Total Snow Flag,Total Precip (mm),Total Precip Flag,Snow on Grnd (cm),Snow on Grnd Flag,Dir of Max Gust (10s deg),Dir of Max Gust Flag,Spd of Max Gust (km/h),Spd of Max Gust Flag,Station ID
0,-79.40,43.63,TORONTO CITY CENTRE,6158359,1980-01-01,1980,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48549
1,-79.40,43.63,TORONTO CITY CENTRE,6158359,1980-01-02,1980,1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48549
2,-79.40,43.63,TORONTO CITY CENTRE,6158359,1980-01-03,1980,1,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48549
3,-79.40,43.63,TORONTO CITY CENTRE,6158359,1980-01-04,1980,1,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48549
4,-79.40,43.63,TORONTO CITY CENTRE,6158359,1980-01-05,1980,1,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48549
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92041,-73.74,45.47,MONTREAL INTL A,7025251,2021-12-27,2021,12,27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,51157
92042,-73.74,45.47,MONTREAL INTL A,7025251,2021-12-28,2021,12,28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,51157
92043,-73.74,45.47,MONTREAL INTL A,7025251,2021-12-29,2021,12,29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,51157
92044,-73.74,45.47,MONTREAL INTL A,7025251,2021-12-30,2021,12,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,51157


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (21,23,27,30) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
temp = all_cities[["Station Name", "Date/Time", "Max Temp (°C)", "Min Temp (°C)", "Total Precip (mm)"]].dropna(subset=["Max Temp (°C)", "Min Temp (°C)", "Total Precip (mm)"], how="all")
toronto = all_cities[all_cities["Station Name"] == "TORONTO CITY CENTRE"]

temp_pivot = toronto.melt(value_vars=["Max Temp (°C)", "Min Temp (°C)", "Total Precip (mm)"], id_vars=['Date/Time']).dropna()
temp_pivot = pd.pivot(temp_pivot, columns="variable", index="Date/Time")
display(temp_pivot)
temp_pivot.to_csv('/content/drive/MyDrive/Data/exports/temp_for_cities.csv')

value                                
variable   Max Temp (°C) Min Temp (°C) Total Precip (mm)
Date/Time                                               
2010-02-02          -1.4          -6.7               0.8
2010-02-03           2.0          -4.7               0.3
2010-02-04           1.6          -8.1               0.0
2010-02-05           0.9          -5.8               0.0
2010-02-06          -4.1         -10.9               0.0
...                  ...           ...               ...
2021-11-13           8.1           4.2               0.0
2021-11-14           5.7           2.4               8.7
2021-11-15           6.9           1.8               0.5
2021-11-16           7.4           1.4               0.0
2021-11-17          13.3           3.7               0.0

[4134 rows x 3 columns]

### 2021/11/25 -  Nova Scotia Nov. 23/24 data

In [ ]:
ns_stations = stations[stations["Province"] == 'NOVA SCOTIA']
station_id_list = ns_stations["Station ID"].dropna().astype(int).to_list()

In [ ]:
li = []

for station_id in station_id_list:
  for year in range(2021,2022):
    url = 'https://climate.weather.gc.ca/climate_data/bulk_data_e.html?format=csv&stationID=' + str(station_id) + '&Year=' + str(year) + '&timeframe=2'
    df = pd.read_csv(url)
    df.insert(0, "Station ID", station_id)
    li.append(df)
  
ns_stations_raw = pd.concat(li, axis=0, ignore_index=True)
ns_stations_raw.to_csv('/content/drive/MyDrive/Data/raw/weather/ns_stations_raw.csv')

In [ ]:
nov23 = ns_stations_raw[ns_stations_raw["Date/Time"].isin(["2021-11-23", "2021-11-24"])].groupby(["Station Name", "Latitude (y)", "Longitude (x)"]).sum().sort_values("Total Precip (mm)", ascending=False).reset_index()
nov23 = nov23[["Station ID", "Longitude (x)", "Latitude (y)", "Station Name", "Total Precip (mm)"]]
nov23 = nov23[nov23["Total Precip (mm)"] != 0.0]

display(nov23)
nov23.to_csv('/content/drive/MyDrive/Data/exports/ns_stations_nov23.csv')

,Station ID,Longitude (x),Latitude (y),Station Name,Total Precip (mm)
0,60652,-60.41,46.66,INGONISH BEACH RCS,266.0
1,213598,-60.05,46.16,SYDNEY A,163.2
2,21322,-60.98,46.37,NORTHEAST MARGAREE (AUT),116.6
3,97336,-61.37,45.66,PORT HAWKESBURY,107.1
4,109156,-59.97,45.92,LOUISBOURG,100.6
5,107076,-62.01,45.49,COLLEGEVILLE AUTO,97.7
6,99496,-60.65,45.92,ESKASONI FIRST NATION AUTOMATIC WEATHER STATION,76.0
7,109236,-60.95,46.65,CHETICAMP HIGHLANDS NATIONAL PARK,71.5
8,104404,-59.96,43.93,SABLE ISLAND A,64.0
9,122186,-60.01,43.93,SABLE ISLAND,61.0


### 2021/11/25 - Newfoundland Nov. 23/24 data




In [ ]:
nl_stations = stations[stations["Province"] == 'NEWFOUNDLAND']
nl_station_id_list = nl_stations["Station ID"].dropna().astype(int).to_list()

In [ ]:
li = []

for station_id in nl_station_id_list:
  for year in range(2021,2022):
    url = 'https://climate.weather.gc.ca/climate_data/bulk_data_e.html?format=csv&stationID=' + str(station_id) + '&Year=' + str(year) + '&timeframe=2'
    df = pd.read_csv(url)
    df.insert(0, "Station ID", station_id)
    li.append(df)
  
nl_stations_raw = pd.concat(li, axis=0, ignore_index=True)
nl_stations_raw.to_csv('/content/drive/MyDrive/Data/raw/weather/nl_stations_raw.csv')

In [ ]:
nl_nov23 = nl_stations_raw[nl_stations_raw["Date/Time"].isin(["2021-11-23", "2021-11-24"])].groupby(["Station Name", "Latitude (y)", "Longitude (x)"]).sum().sort_values("Total Precip (mm)", ascending=False).reset_index()
nl_nov23 = nl_nov23[["Station ID", "Longitude (x)", "Latitude (y)", "Station Name", "Total Precip (mm)"]]
nl_nov23 = nl_nov23[nl_nov23["Total Precip (mm)"] != 0.0]

nl_nov23.to_csv('/content/drive/MyDrive/Data/exports/nl_stations_nov23.csv')
display(nl_nov23)

,Station ID,Longitude (x),Latitude (y),Station Name,Total Precip (mm)
0,122666,-59.15,47.57,PORT AUX BASQUES,165.1
1,13172,-57.33,48.17,BURNT POND,54.0
2,215072,-57.04,53.68,CARTWRIGHT A,45.8
3,110156,-60.56,53.41,HAPPY VALLEY GOOSE BAY,41.7
4,13554,-60.42,53.32,GOOSE A,41.6
5,207834,-55.85,52.30,MARY'S HARBOUR A,39.8
6,18068,-57.10,51.02,FEROLLE POINT (AUT),22.5
7,95214,-58.57,48.56,STEPHENVILLE RCS,22.4
8,96696,-56.07,51.39,ST. ANTHONY A,21.5
9,105518,-58.55,48.54,STEPHENVILLE A,20.2


### 2021/11/28 - All 2021 data

In [ ]:
all_stations_2021 = pd.read_csv('/content/drive/MyDrive/Data/raw/weather/all_stations_raw.csv')

recent = (
    all_stations_2021[all_stations_2021["Date/Time"] >= "2021-11-27"]
    .dropna(subset=["Total Precip (mm)"])
    .sort_values("Total Precip (mm)", ascending=False)
)

recent = recent[recent["Total Precip (mm)"] != 0.0]
recent = recent[["Station Name", "Climate ID", "Date/Time", "Total Precip (mm)"]]

display(recent)
recent.head(20).to_csv("/content/drive/MyDrive/Data/exports/weather/weather-nov28.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5,10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Station Name,Climate ID,Date/Time,Total Precip (mm)
4345,PORT RENFREW,1016335,2021-11-27,193.0
17850,CAPE BEALE LIGHT,1031316,2021-11-27,128.6
19310,NITINAT RIVER HATCHERY,1035612,2021-11-27,121.6
64571,HOPE AIRPORT,1113543,2021-11-28,119.0
20040,PACHENA POINT,1035940,2021-11-27,117.6
...,...,...,...,...
267876,HIGH LEVEL,3073148,2021-11-28,0.1
351460,THUNDER BAY CS,6048268,2021-11-27,0.1
479211,LA TUQUE,707DBD4,2021-11-28,0.1
205096,EDMONTON STONY PLAIN CS,301A001,2021-11-28,0.1


In [ ]:
"all_stations_nov23 = all_stations_2021[["Station Name", "Station ID", 'Date/Time', "Total Precip (mm)"]].sort_values("Total Precip (mm)", ascending=False).dropna()
all_stations_nov23 = all_stations_nov23[all_stations_nov23["Total Precip (mm)"] != 0.0].head(20)

all_stations_nov23.to_csv('/content/drive/MyDrive/Data/exports/record_rainfalls.csv')

### 2021/11/28 - All 2020 data

In [ ]:
li = []

for station_id in all_stations_list:
    url = 'https://climate.weather.gc.ca/climate_data/bulk_data_e.html?format=csv&stationID=' + str(station_id) + '&Year=2020&timeframe=2'
    df = pd.read_csv(url)
    df.insert(0, "Station ID", station_id)
    li.append(df)
  
all_stations_raw_2020 = pd.concat(li, axis=0, ignore_index=True)
all_stations_raw_2020.to_csv('/content/drive/MyDrive/Data/raw/weather/all_stations_raw_2020.csv')

In [ ]:
all_stations_2020 = all_stations_raw_2020[["Station Name", "Station ID", 'Date/Time', "Total Precip (mm)"]].sort_values("Total Precip (mm)", ascending=False).dropna()
all_stations_2020 = all_stations_2020[all_stations_2020["Total Precip (mm)"] != 0.0].head(20)

all_stations_2020.to_csv('/content/drive/MyDrive/Data/exports/record_rainfalls_2020.csv')
display(all_stations_2020)

,Station Name,Station ID,Date/Time,Total Precip (mm)
19062,NITINAT RIVER HATCHERY,260,2020-01-31,299.4
98057,PLEASANT CAMP,1504,2020-12-01,280.0
313109,RIVERS PETTAPIECE,3457,2020-06-28,238.9
59294,POINT ATKINSON,844,2020-01-03,221.4
507245,MECHANIC SETTLEMENT,45090,2020-12-01,171.1
315635,MARQUETTE,3619,2020-05-23,167.8
58224,N VANC GROUSE MTN RESORT,823,2020-01-31,163.5
307619,BRANDON A,50821,2020-06-28,155.5
58200,N VANC GROUSE MTN RESORT,823,2020-01-07,154.0
307985,BRANDON RCS,49909,2020-06-28,152.3


### 2021/11/28 - Data since 2000

In [ ]:
li = []

for year in range(2000, 2022):
  for station_id in all_stations_list:
    url = 'https://climate.weather.gc.ca/climate_data/bulk_data_e.html?format=csv&stationID=' + str(station_id) + '&Year=' + str(year) + '&timeframe=2'
    df = pd.read_csv(url)
    df.insert(0, "Station ID", station_id)
    li.append(df)
  
all_stations_raw_2000 = pd.concat(li, axis=0, ignore_index=True)
all_stations_raw_2000.to_csv('/content/drive/MyDrive/Data/raw/weather/all_stations_raw_2000-2021.csv')

KeyboardInterrupt: ignored

### 2022/01/06 - Abbotsford Rainfall in November

In [ ]:
lis = []

for year in range(1980, 2022):
  url = 'https://climate.weather.gc.ca/climate_data/bulk_data_e.html?format=csv&stationID=50308&Year=' + str(year) + '&timeframe=2'
  df = pd.read_csv(url)
  lis.append(df)
  
abby_raw = pd.concat(lis, axis=0, ignore_index=True)
display(abby_raw)

,Longitude (x),Latitude (y),Station Name,Climate ID,Date/Time,Year,Month,Day,Data Quality,Max Temp (°C),Max Temp Flag,Min Temp (°C),Min Temp Flag,Mean Temp (°C),Mean Temp Flag,Heat Deg Days (°C),Heat Deg Days Flag,Cool Deg Days (°C),Cool Deg Days Flag,Total Rain (mm),Total Rain Flag,Total Snow (cm),Total Snow Flag,Total Precip (mm),Total Precip Flag,Snow on Grnd (cm),Snow on Grnd Flag,Dir of Max Gust (10s deg),Dir of Max Gust Flag,Spd of Max Gust (km/h),Spd of Max Gust Flag
0,-122.36,49.03,ABBOTSFORD A,1100031,1980-01-01,1980,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-122.36,49.03,ABBOTSFORD A,1100031,1980-01-02,1980,1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-122.36,49.03,ABBOTSFORD A,1100031,1980-01-03,1980,1,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-122.36,49.03,ABBOTSFORD A,1100031,1980-01-04,1980,1,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-122.36,49.03,ABBOTSFORD A,1100031,1980-01-05,1980,1,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15336,-122.36,49.03,ABBOTSFORD A,1100031,2021-12-27,2021,12,27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15337,-122.36,49.03,ABBOTSFORD A,1100031,2021-12-28,2021,12,28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15338,-122.36,49.03,ABBOTSFORD A,1100031,2021-12-29,2021,12,29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15339,-122.36,49.03,ABBOTSFORD A,1100031,2021-12-30,2021,12,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
abbotsford = abby_raw[abby_raw["Station Name"].str.contains('ABBOTSFORD')]
abbotsford = abbotsford[["Station Name", "Date/Time", "Year", "Month", "Day", "Total Precip (mm)"]].dropna()
pivot = pd.pivot_table(abbotsford, columns=["Month"], index="Year", values="Total Precip (mm)", aggfunc=sum)

display(pivot)
pivot.to_csv('/content/drive/MyDrive/Data/exports/weather/abbotsford_2012.csv')

Month,1,2,3,4,5,6,7,8,9,10,11,12
Year,,,,,,,,,,,,
2012,NaN,NaN,NaN,NaN,NaN,58.6,52.7,3.8,6.6,261.8,160.2,161.2
2013,135.7,102.8,214.9,144.3,103.3,81.0,3.8,25.7,122.5,63.1,217.5,118.0
2014,137.9,100.3,283.1,121.6,123.2,42.2,35.1,7.6,97.7,230.8,232.5,214.2
2015,173.8,111.4,187.9,52.9,8.6,12.1,51.9,43.3,70.6,102.1,185.6,235.0
2016,166.7,194.6,157.2,71.7,58.0,46.2,30.9,14.5,69.5,271.3,324.9,144.4
2017,111.5,188.7,309.1,173.6,102.9,46.0,4.4,3.2,75.5,186.6,239.8,199.6
2018,253.9,136.7,122.8,143.5,20.4,40.3,20.0,8.0,153.1,120.6,256.2,225.9
2019,160.9,94.7,42.1,145.3,52.9,28.0,50.2,20.5,159.7,132.2,102.5,223.1
2020,359.7,150.7,123.3,49.6,111.1,75.9,50.1,43.7,51.7,124.9,256.4,296.7


### 2022/01/18 - Vancouver Harbour

In [3]:
lis = []

for year in range(1980, 2022):
  url = 'https://climate.weather.gc.ca/climate_data/bulk_data_e.html?format=csv&stationID=888&Year=' + str(year) + '&Month=11&Day=14&timeframe=2'
  df = pd.read_csv(url)
  lis.append(df)
  
van = pd.concat(lis, axis=0, ignore_index=True)
van.to_csv('/content/drive/MyDrive/Data/raw/weather/van_historical.csv')

In [5]:
nov14 = van[van["Date/Time"].str.contains("11-14")]
nov15 = van[van["Date/Time"].str.contains("11-15")]
nov = pd.concat([nov14, nov15])
nov_sum = nov[['Station Name', 'Year', 'Total Precip (mm)']].groupby(['Station Name', 'Year']).sum().reset_index()

display(nov_sum)

,Station Name,Year,Total Precip (mm)
0,VANCOUVER HARBOUR CS,1980,0.8
1,VANCOUVER HARBOUR CS,1981,20.4
2,VANCOUVER HARBOUR CS,1982,31.2
3,VANCOUVER HARBOUR CS,1983,101.2
4,VANCOUVER HARBOUR CS,1984,0.0
5,VANCOUVER HARBOUR CS,1985,21.6
6,VANCOUVER HARBOUR CS,1986,9.4
7,VANCOUVER HARBOUR CS,1987,21.9
8,VANCOUVER HARBOUR CS,1988,0.0
9,VANCOUVER HARBOUR CS,1989,0.8
